In [187]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

dateparse = lambda x: pd.datetime.strptime(x, '%d-%m-%Y %H')

In [188]:
### 1- First we need to estimate the daily phone call densities
### 2- Second we will aplly the model to those densities 
### 3- And finally collect the data in 81 col 365 rows dataframe 
###    in wich every cell is the pop density in that province in that day

In [195]:
### 1
cdr_data_path = '/home/lorentz/Desktop/Datasets/D4R/Dataset1_Voice_New/Dataset 1_2017'
sms_data_path = '/home/lorentz/Desktop/Datasets/D4R/Dataset1_SMS_New/Dataset 1_SMS_2017'
base_station_location_path = '/home/lorentz/Desktop/Datasets/D4R/Base_Station_Location.txt'
overlap_shapefile = '/home/lorentz/Desktop/Datasets/D4R/overlap_file.shp'

### Load base_station data
cell_location = pd.read_csv(base_station_location_path, encoding = 'utf-8', error_bad_lines=False)
### Load VORONOI overlap shapefile
shapefile = gpd.read_file(overlap_shapefile)
### Check for tower duplicates and assign the same site id to the tower with the same coordinates
duplicates = cell_location.groupby(['MX_LAT1','MX_LAT2','MX_LAT3','MX_LONG1','MX_LONG2','MX_LONG3']).apply(lambda x:[(x.iloc[i]['BTS_ID'],x.iloc[0]['BTS_ID']) for i in range(len(x))] )
dizionario_duplicates = dict([item for sublist in duplicates.values for item in sublist])
### Inizialize empty dataframe in which the sigmas will be stored
df_sigma_t = pd.DataFrame(columns=list(shapefile.drop_duplicates('NAME_1')['NAME_1'].sort_values()),
                          data=None)
df_sigma_t.index.name = 'Time_day'
df_sigma_r = pd.DataFrame(columns=list(shapefile.drop_duplicates('NAME_1')['NAME_1'].sort_values()),
                          data=None)
df_sigma_r.index.name = 'Time_day'

### Loop over the 12 monthly voice call files
days = 0
for i in range(1,13,1):
    if i <10:
        cdr_data = pd.read_csv(cdr_data_path+'0'+str(i)+'.txt', encoding = 'utf-8', parse_dates=[0], date_parser=dateparse)
    elif i>=10:
        cdr_data = pd.read_csv(cdr_data_path+str(i)+'.txt', encoding = 'utf-8', parse_dates=[0], date_parser=dateparse)
    cdr_data = cdr_data[cdr_data.TIMESTAMP.apply(lambda x:x.hour>=20 or x.hour<=7)]
    cdr_data = cdr_data[cdr_data.TIMESTAMP.apply(lambda x: x.month==i)]
    
    cdr_data['OUTGOING_SITE_ID'] = cdr_data['OUTGOING_SITE_ID'].map(dizionario_duplicates)
    group_day = cdr_data.groupby(cdr_data.set_index('TIMESTAMP').index.date)
    days += len(group_day)
    for name, group in group_day:
    
        outgoing_calls_t = group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_CALLS'].mean()-\
                       group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_REFUGEE_CALLS'].mean()
        outgoing_calls_r = group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_REFUGEE_CALLS'].mean()

        new_df = pd.concat([ outgoing_calls_t, outgoing_calls_r],
                            axis = 1).fillna(0)
        new_df.columns =  ['calls_outgoing_t','calls_outgoing_r']
        dict_calls = new_df.to_dict()

        for key in dict_calls.keys():
            shapefile[key] = shapefile['BTS_ID'].map(dict_calls[key])
        shapefile = shapefile.fillna(0)
        shapefile['calls_outgoing_density_t'] = shapefile['area_fract']*shapefile['calls_outgoing_t']/shapefile['area_voron']
        shapefile['calls_outgoing_density_r'] = shapefile['area_fract']*shapefile['calls_outgoing_r']/shapefile['area_voron']

        df_sigma_c = shapefile.groupby('NAME_1')[['calls_outgoing_density_t',
                                                  'calls_outgoing_density_r']].sum()
        df_sigma_c = df_sigma_c.sort_index()
        df_sigma_c1 = df_sigma_c[['calls_outgoing_density_t']].T.reset_index(drop=True)
        df_sigma_c1['Time_day'] = name
        df_sigma_c1.set_index('Time_day',inplace=True)
        df_sigma_t = df_sigma_t.append(df_sigma_c1)
        
        df_sigma_c2 = df_sigma_c[['calls_outgoing_density_r']].T.reset_index(drop=True)
        df_sigma_c2['Time_day'] = name
        df_sigma_c2.set_index('Time_day',inplace=True)
        df_sigma_r = df_sigma_r.append(df_sigma_c2)

    print('processing ...{}%'.format(round(i/0.12),1),end='\r')
print('processing ... COMPLETE',end='\r')

In [196]:
### 2
df_prov_sigma_t = df_sigma_t.copy()
df_prov_sigma_r = df_sigma_r.copy()


alpha_t = 1428 #+-4
beta_t = 0.801 #+-0.004
# sigma_t = 16.5 #+-0.5


alpha_r = 280. #+-10
beta_r = 0.80 #+-0.05
# sigma_r = 9.4 #+-0.8  

sigma_pop_t = alpha_t*df_prov_sigma_t**beta_t
sigma_pop_r = alpha_r*df_prov_sigma_r**beta_r

sigma_pop_t.to_csv('daily_sigma_pop_t.csv')
sigma_pop_r.to_csv('daily_sigma_pop_r.csv')

In [197]:
# Check on the predicted total population
ttt = sigma_pop_t.mean(axis=0).reset_index()
tmp = shapefile[['NAME_1','area','area_fract']]
ttmp = tmp.merge(ttt, on='NAME_1').drop_duplicates('NAME_1')
ttmp['p_area'] = ttmp['area']/ttmp['area_fract']
ttmp['pop'] = ttmp['p_area']*ttmp[0]
ttmp.sort_values('pop',ascending=False).sum()

In [210]:
### 1.b
### In a completely similar way we can do for the densities at the base station location level

cdr_data_path = '/home/lorentz/Desktop/Datasets/D4R/Dataset1_Voice_New/Dataset 1_2017'
sms_data_path = '/home/lorentz/Desktop/Datasets/D4R/Dataset1_SMS_New/Dataset 1_SMS_2017'
base_station_location_path = '/home/lorentz/Desktop/Datasets/D4R/Base_Station_Location.txt'
overlap_shapefile = '/home/lorentz/Desktop/Datasets/D4R/overlap_file.shp'

### Load base_station data
cell_location = pd.read_csv(base_station_location_path, encoding = 'utf-8', error_bad_lines=False)
### Load VORONOI overlap shapefile
shapefile = gpd.read_file(overlap_shapefile)
### Check for tower duplicates and assign the same site id to the tower with the same coordinates
duplicates = cell_location.groupby(['MX_LAT1','MX_LAT2','MX_LAT3','MX_LONG1','MX_LONG2','MX_LONG3']).apply(lambda x:[(x.iloc[i]['BTS_ID'],x.iloc[0]['BTS_ID']) for i in range(len(x))] )
dizionario_duplicates = dict([item for sublist in duplicates.values for item in sublist])
### Inizialize empty dataframe in which the simas will be stored
df_sigma_t = pd.DataFrame(columns=list(shapefile.drop_duplicates('BTS_ID')['BTS_ID'].sort_values()),
                          data=None)
df_sigma_t.index.name = 'Time_day'
df_sigma_r = pd.DataFrame(columns=list(shapefile.drop_duplicates('BTS_ID')['BTS_ID'].sort_values()),
                          data=None)
df_sigma_r.index.name = 'Time_day'

### Loop over the 12 monthly voice call files
days = 0
for i in range(1,13,1):
    if i <10:
        cdr_data = pd.read_csv(cdr_data_path+'0'+str(i)+'.txt', encoding = 'utf-8', parse_dates=[0], date_parser=dateparse)
    elif i>=10:
        cdr_data = pd.read_csv(cdr_data_path+str(i)+'.txt', encoding = 'utf-8', parse_dates=[0], date_parser=dateparse)
    cdr_data = cdr_data[cdr_data.TIMESTAMP.apply(lambda x: x.hour>=20 or x.hour<=7)].reset_index(drop=True)
    cdr_data = cdr_data[cdr_data.TIMESTAMP.apply(lambda x: x.month==i)]

        #### -> un modo interessante per inserire in modo semplice il conteggio delle ore della sera
####    come ore del giorno dopo o quelle del mattino come ore del giorno prima, é:
####    Inserire qui una riga in cui si aggiunge o sottrae a tutti i timestamp 4 o 7 ore
####    É importante qui perché poi raggruppiamo per giorno e non saprei ocme farlo in modo cosí semplice


    cdr_data['OUTGOING_SITE_ID'] = cdr_data['OUTGOING_SITE_ID'].map(dizionario_duplicates)
    group_day = cdr_data.groupby(cdr_data.set_index('TIMESTAMP').index.date)
    days += len(group_day)
    for name, group in group_day:
    
        outgoing_calls_t = group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_CALLS'].mean()-\
                       group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_REFUGEE_CALLS'].mean()
        outgoing_calls_r = group.groupby(['OUTGOING_SITE_ID'])['NUMBER_OF_REFUGEE_CALLS'].mean()

        new_df = pd.concat([ outgoing_calls_t, outgoing_calls_r],
                            axis = 1).fillna(0)
        new_df.columns =  ['calls_outgoing_t','calls_outgoing_r']
        dict_calls = new_df.to_dict()

        for key in dict_calls.keys():
            shapefile[key] = shapefile['BTS_ID'].map(dict_calls[key])
        shapefile = shapefile.fillna(0)
        shapefile['calls_outgoing_density_t'] = shapefile['calls_outgoing_t']/shapefile['area_voron']
        shapefile['calls_outgoing_density_r'] = shapefile['calls_outgoing_r']/shapefile['area_voron']

        df_sigma_c = shapefile.groupby('BTS_ID')[['calls_outgoing_density_t',
                                                  'calls_outgoing_density_r']].sum()
        df_sigma_c = df_sigma_c.sort_index()
        df_sigma_c1 = df_sigma_c[['calls_outgoing_density_t']].T.reset_index(drop=True)
        df_sigma_c1['Time_day'] = name
        df_sigma_c1.set_index('Time_day',inplace=True)
        df_sigma_t = df_sigma_t.append(df_sigma_c1)
        
        df_sigma_c2 = df_sigma_c[['calls_outgoing_density_r']].T.reset_index(drop=True)
        df_sigma_c2['Time_day'] = name
        df_sigma_c2.set_index('Time_day',inplace=True)
        df_sigma_r = df_sigma_r.append(df_sigma_c2)
    print('processing ...{}%'.format(round(i/0.12),1),end='\r')
print('processing ... COMPLETE',end='\r')

In [211]:
### 2.b
df_cell_sigma_t = df_sigma_t.copy()
df_cell_sigma_r = df_sigma_r.copy()

sigma_cell_pop_t = alpha_t*df_cell_sigma_t**beta_t
sigma_cell_pop_r = alpha_r*df_cell_sigma_r**beta_r
sigma_cell_pop_t.to_csv('daily_cell_sigma_pop_t.csv')
sigma_cell_pop_r.to_csv('daily_cell_sigma_pop_r.csv')

In [212]:
# Check on the predicted total population
ttt = sigma_cell_pop_t.mean(axis=0).reset_index()
tmp = shapefile[['BTS_ID','area','area_fract','NAME_1','area_voron']]
ttmp = ttt.merge(tmp, on='BTS_ID',how='left').drop_duplicates('BTS_ID')
ttmp['pop'] = ttmp['area_voron']*ttmp[0]
ttmp.sum()

In [214]:
### Check if there is only one row per day
tmp = df_sigma_t.reset_index().groupby('Time_day').count()
tmp[tmp[tmp.columns.values[1]]>1]

BTS_ID,5053271,5053276,5053277,5053278,5053281,5053283,5053284,5053285,5053287,5053290,...,5232953,5232955,5232960,5232963,5232965,5232968,5232969,5232978,5232979,5232980
Time_day,,,,,,,,,,,,,,,,,,,,,
